In [12]:
from pathlib import Path
import torch
import zuko

In [15]:
dir_path = Path.cwd()

# /bayes-air-atrds-attempt-8/checkpoints/nsf_730
ckpt_path = dir_path / 'bayes-air-atrds-attempt-8/checkpoints/nsf_730/checkpoint_0.pt'

ckpt_dict = torch.load(ckpt_path)
# print(ckpt_dict)

guide_state_dict = ckpt_dict['guide']

guide = zuko.flows.NSF(
    features=1,
    context=5,
    hidden_features=(16, 16),
    bins=4,
) #this is the only one that really works??

samples = guide(torch.tensor([1.0, 0.0, 0.0, 0.0, 1.0])).sample((1000,))

def transform_sample(sample):
    # return .02+.015*torch.tanh(a*sample)
    return .004 * sample + .02

print(transform_sample(samples).mean())


tensor(0.0260)
